# Requriement
## Envinronment
- 클러스터, 클라이언트 갯수를 랜덤으로 생성하고 초기화
- 각 클러스터에 스트림을 제공하면 Utility metric을 계산해서 알려줌
- dataset으로부터 trace를 가져와서 CH의 REMB를 결정

In [276]:
import numpy as np
import os

RANDOM_SEED = None
RANDOM_SEED = 10

class Environment:
    # -- class method --
    def get_trace_files(trace_dir):
        filepaths = []
        for filename in os.listdir(trace_dir):
            if filename[0] != ".":
                trace_path = trace_dir + filename
                filepaths.append(trace_path)
        return filepaths
    
    # -- instance method --
    def __init__(self,
                 random_seed=RANDOM_SEED,
                 num_of_cluster_low=1,
                 num_of_cluster_high=100,
                 num_of_client_low=1,
                 num_of_client_high=30,
                 trace_year_month=None,
                 trace_dir="./dataset/cooked/",
                 trace_type="youtube.com"
                ):
        
        # -- random seed --
        self.random_seed = random_seed
        np.random.seed(self.random_seed)
        
        # -- network configuration --
        # 클러스터 갯수를 랜덤으로 지정
        self.num_of_cluster = np.random.randint(low=num_of_cluster_low, high=num_of_cluster_high)
        # 각 클러스터 안에 포함된 클라이언트의 수
        self.num_of_client_in_each_cluster = np.random.randint(low=num_of_client_low, high=num_of_client_high, size=self.num_of_cluster)
        
        # -- trace --
        # cooked directory에 있는 모든 트레이스 파일
        self.trace_files_all = Environment.get_trace_files(trace_dir)
        # trace file 중에서 클러스터 갯수만큼 랜덤으로 trace들을 선택함
        self.selected_traces = self.get_random_traces(self.trace_files_all, self.num_of_cluster)
        # get_remb_of_cluster_head() 함수를 부를 때 마다 bandwidth를 변경하기 위해서 존재함
        self.trace_iterator = 0
        
    def get_random_traces(self, traces, size):
        random_trace_files = np.random.choice(traces, size=size, replace=False)
        
        traces = []
        for path in random_trace_files:
            bws = []
            with open(path, 'rb') as f:
                for line in f:
                    line = line.decode()
                    throughput = int(line)
                    bws.append(throughput * 8 / 1000000) # Convert from Byte to Mbps
            traces.append(bws)
                    
        return traces
    
    def get_remb_of_cluster_head(self):
        idx = self.trace_iterator
        remb_list = []
        for i in range(self.num_of_cluster):
            idx_r = idx % len(self.selected_traces[i])
            bw = self.selected_traces[i][idx_r]
            remb_list.append(bw)
        self.trace_iterator += 1
        return remb_list
    
    def set_bitrate_of_streams():
        pass
        
    def p(self):
        print(self.num_of_cluster)
        print(self.num_of_client_in_each_cluster)
        print(sum(self.num_of_client_in_each_cluster))
        
        
env = Environment()
env.p()

10
[ 5 16  1 18 28 29 26 17 18 27]
185
['./dataset/cooked/trace_201704_215332_http---www.youtube.com', './dataset/cooked/trace_201702_5559_http---www.youtube.com', './dataset/cooked/trace_201701_26845_http---www.youtube.com', './dataset/cooked/trace_201703_10468_http---www.youtube.com', './dataset/cooked/trace_201704_32508_http---www.youtube.com', './dataset/cooked/trace_201701_862_http---www.youtube.com', './dataset/cooked/trace_201701_625944_http---www.youtube.com', './dataset/cooked/trace_201702_7238_http---www.youtube.com', './dataset/cooked/trace_201703_12403_http---www.youtube.com', './dataset/cooked/trace_201703_232418_http---www.youtube.com', './dataset/cooked/trace_201702_9632_http---www.youtube.com', './dataset/cooked/trace_201701_598744_http---www.youtube.com', './dataset/cooked/trace_201703_811992_http---www.youtube.com', './dataset/cooked/trace_201704_607790_http---www.youtube.com', './dataset/cooked/trace_201701_799434_http---www.youtube.com', './dataset/cooked/trace_2017

In [272]:
a = env.get_remb_of_cluster_head()
print(a)

[4.371344, 5.083928, 2.20748, 5.174328, 5.41904, 2.449376, 5.273504, 5.257552, 6.052704, 2.324288]
